In [546]:
# Aux
import numpy as np
from scipy.stats import norm
from scipy.linalg import svd
import matplotlib.pyplot as plt

# Model stuff
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import QuantileRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [547]:
synth_gauss = {
    'n': 10000,                            # n_train -> 2500
    'd': [50],                             # data dimensions
    'sigma2': 1,                           # data variance
    'frac_important_features': 0.1,        # fraction of relevant features
    'sigma2_eps': 0.1,                     # label noise
    'corr': 0.0,                           # correlation between features
    'fit_sgd': False,                      # whether linear model should be fitted with sgd
    'n_shadow_models': 200,                # number of shadow models to train for LRT test versions
    'frac': 0.85,                          # fraction of samples used for resampling scheme within shadow model train pipeline
    'epsd': 1e-45,                         # constant to ensure stable loss evaluations
    's': 0                                 # target score in logit space
}

In [548]:
def gen_lin_data(d:int, synth_gauss: dict):
    
    '''
    This method generates data from the data generating process described above. 
    ---------------------------------------------------------------------------
        Input: data dimension (int), snyth gauss (dictionary)
        Output: X, Y (np.arrays); training set size (int)
    '''
    
    n = synth_gauss['n']
    n_train = int(n/4)
    print(f'Number of training samples: {n_train}')

    n_train_half = int(2*int(n/2))
    sigma2 = synth_gauss['sigma2']
    frac_important_features = synth_gauss['frac_important_features']
    d_relevant = int(np.floor(d * frac_important_features))
    mu = np.zeros(d)
    Sigma2 = np.eye(d) * sigma2
    Sigma2 = np.ones((d, d)) * synth_gauss['corr'] + np.eye(d) * sigma2


    ### true parameter vector ###
    beta0 = np.zeros(d)
    beta = np.random.random(d) * 2 - 1                            # Coefficient randomly chosen from [-1,1]^d 
    quantile = np.quantile(np.abs(beta), 1 - frac_important_features)
    indeces = np.where(np.abs(beta) >= quantile)[0]               # Determine indeces of relevant coefficients
    
    beta0[0:d_relevant] = beta[indeces]
    beta1 = beta0 / np.linalg.norm(beta0, ord=2)                  # Normalize to have unit vector 1

    ### error variance parameters & distribution of errors ###
    sigma2_eps = synth_gauss['sigma2_eps']
    sigma_eps = np.sqrt(sigma2_eps)
    eps = np.random.normal(0, sigma_eps, n)                       # Parameterzied in terms of sigma (not sigma2)

    ### data generation ###
    X = np.random.multivariate_normal(mu, Sigma2, n)              # Generate & standardize data
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)

    score = X @ beta1 + eps                                       # Generate labels
    prob = 1/(1+np.exp(-score))
    Y_disc = (prob > 0.5) * 1
    
    return X, Y_disc, n_train

In [726]:
epsd = synth_gauss['epsd']
for i in synth_gauss['d']:
    
    ######################
    ### GENERATE DATA ###
    #####################
    print(f'Computing for dimension: {i}')
    
    X, Y_disc, n_train = gen_lin_data(d=i, synth_gauss=synth_gauss)
    
    # split data
    X, X_prime, Y, Y_prime = train_test_split(X.astype(float), 
                                              Y_disc.astype(float), 
                                              test_size=0.5,
                                              random_state=10)

    Y_train = Y[0:n_train]
    Y_test = Y[n_train::]
    X_train = X[0:n_train, :]
    X_test = X[n_train::, :]
    n_test = X_test.shape[0]
    
    ######################
    ###    FIT MODEL   ###
    ######################
    
    if synth_gauss['fit_sgd']:
        clf = SGDClassifier(loss='log', penalty='none', fit_intercept=True, max_iter=1500, tol=1e-6)
        clf.fit(X_train, Y_train)
    else:
        clf = LogisticRegression(penalty='none', fit_intercept=True, max_iter=1500)
        clf.fit(X_train, Y_train)
        
    print('training set accuracy:', clf.score(X_train, Y_train))
    print('test set accuracy:', clf.score(X_test, Y_test))
    
    f_train = np.log(clf.predict_proba(X_train)[:,1] + epsd) - np.log(clf.predict_proba(X_train)[:,0] + epsd)
    w_train = clf.coef_[0]
    
    
    k = 3
    t = 0
    
    cov = (X_train.T @ X_train) / (X_train.shape[0] - 1)
    eig_values, eig_vectors = np.linalg.eig(cov)
    idx = np.argsort(eig_values, axis=0)[::-1]
    sorted_eig_vectors = eig_vectors[:, idx]
    W = sorted_eig_vectors[:, :k]
    Z = np.dot(X, W)
    Xhat = np.dot(Z, W.T)
                      
    x = X_train[0,:]
    z = Z[0,:]
    xhat = Xhat[0,:]
    rx = x - W @ z
    ry = synth_gauss['s'] - f_train[0]
    rxy = ry + w_train @ rx
    w_tilde = W.T @ w_train

    delta_z = (ry / np.linalg.norm(w_tilde, 2)**2) * w_tilde
    print('length delta', np.linalg.norm(delta,2))
    cf_z = z + delta_z
    cf = x + W @ delta_z
    print(cf.shape)
    cf = cf.reshape(1,-1)
    print(clf.predict_proba(x.reshape(1,-1))[0][1])
    print(clf.predict_proba(cf)[0][1])

Computing for dimension: 50
Number of training samples: 2500
training set accuracy: 0.9092
test set accuracy: 0.9056
length delta 0.09664016587839812
(50,)
0.007172721044610176
0.4999999999999999
